In [2]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import os
import json
from tools.data_utils import *
from tools.optimization import EarlyStoppingCallback, evaluate_model
import matplotlib.pyplot as plt
import argparse
import time
from tools.NPmodels import *
from tools.NPtrain import process_batch, NeuralProcessTrainer, evaluate_np
from tabulate import tabulate
import torch.optim.lr_scheduler as lr_scheduler
import time

In [3]:
def custom_collate_fn(batch):
    images = [sample['image'] for sample in batch]
    pins = [sample['pins'] for sample in batch]
    outputs = [sample['outputs'] for sample in batch]

    return {
        'image': torch.stack(images, dim=0),
        'pins': pins,
        'outputs': outputs}

In [4]:
def run_pipeline_ci_np(train_loader, val_loader, 
                    test_loader, input_channel, epochs, val_every_epoch, config, device, num_runs=3, print_freq=2):
    test_losses = []
    # partial_percent = config['partial_percent']
    experiment_id = int(time.time())
    best_val_loss_NP = float('inf')
    # config['experiment_id'] = experiment_id
    experiment_id = config['experiment_id']

    r_dim = np_config.r_dim
    h_dim = np_config.h_dim
    z_dim = np_config.z_dim
    lr = config['best_lr']

    # Create storage directory and store the experiment configuration
    if not os.path.exists(f'./results/neural_processes/{experiment_id}'):
        os.makedirs(f'./results/neural_processes/{experiment_id}')
    with open(f"./results/neural_processes/{experiment_id}/config_np.json", "w") as outfile: 
        json.dump(config, outfile)
        
    global_val_loss = float('inf')
    
    for run in range(num_runs):
        count = 0
        GP_test_losses = []
        
        early_stopping = EarlyStoppingCallback(patience=5, min_delta=0.001)
        model = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        np_trainer = NeuralProcessTrainer(device, model, optimizer, early_stopping, experiment_id, print_freq=print_freq)          

        np_trainer.train(train_loader, val_loader, epochs)
        if np_trainer.best_val_loss <= global_val_loss:
            global_val_loss = np_trainer.best_val_loss 
            torch.save(np_trainer.neural_process.state_dict(), f'./results/neural_processes/{experiment_id}' + f'/best_{args.dataset}_np.pt')
        count += 1

    return experiment_id


# Function to run the pipeline and save data
def run_and_save_pipeline_np(train_loader, val_loader, test_loader, input_channel, epochs, val_every_epoch, config, num_runs, device):
    test_partial_percents = [0.25, 0.5, 0.75, 1]
    test_losses = []
    r2_list = []
    table = []
    table.append(['Dataset', 'Mode', 'd', 'n_pins', 'LR', 'PLP', 'MSE error', 'R2'])
    experiment_id = run_pipeline_ci_np(train_loader, val_loader, 
                    test_loader, input_channel, epochs, val_every_epoch, config, device, num_runs)
    # Run final testing
    model = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
    # MSE
    model.load_state_dict(torch.load(f'./results/neural_processes/{experiment_id}/best_{args.dataset}_np.pt'))
    for partial_percent in test_partial_percents:
        test_loss, r2 = evaluate_np(model, test_loader, device, partial_percent=partial_percent)
        print(f"pp: {partial_percent} MSE loss: {test_loss} R2 score: {r2}")
        table.append([args.dataset, args.mode, args.d, args.n_pins, config['best_lr'], partial_percent, test_loss, r2])
        test_losses.append(test_loss)
        r2_list.append(r2)
    table = tabulate(table, headers='firstrow', tablefmt='fancy_grid', showindex=True)
    print(table)
    with open('./results/neural_processes/table.txt', 'a') as f:
        f.write('\n')
        f.write(table + '\n')  # Add a newline character after writing the table
        f.write('\n')  # Add an additional newline character for separation
    print("saved")
    return test_losses, r2_list, experiment_id

In [5]:
class Args():
    dataset = "PinMNIST"
    n = 1000
    mode = "mesh"
    d = 10
    n_pins = 10
    r = 3
    partial_percent = 0.8
    # Set your hyperparameters
    epochs = 1000
    batch_size = 50
    learning_rate = 1e-4
    val_every_epoch = 10
    num_runs = 1
    seed = 4
    
class NP_config():
    r_dim = 512
    h_dim = 512
    z_dim = 512
    lr = 4e-5
    epochs = 100

np_config = NP_config()
args = Args()

In [6]:
# Set a random seed for PyTorch
seed = 4  # You can use any integer value as the seed
torch.manual_seed(seed)
# Set a random seed for NumPy (if you're using NumPy operations)
np.random.seed(seed)

 # Choose datasets
dataset = args.dataset 
n = args.n
mode = args.mode
d = args.d
n_pins = args.n_pins
r = args.r
partial_percent = args.partial_percent

# Set your hyperparameters
epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.learning_rate
val_every_epoch = args.val_every_epoch
num_runs = args.num_runs
   
config = {}
# np_config = {"batch_size": 32,
#              "r_dim": 512,
#              "h_dim": 512,
#              "z_dim": 512,
#              "lr": 4e-5,
#              "epochs": 100
#             }

# config = vars(args)
config = {"experiment_id":0}

input_channel = 1 if dataset == "PinMNIST" else 3
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# if feature_extracted:
#     folder = f"{dataset}_ddpm"
# else:
folder = f"{dataset}"

if dataset == "PinMNIST":
    if mode == "mesh":
        data_folder = f"./data/{folder}/mesh_{d}step_{28}by{28}pixels_{r}radius_{seed}seed"
    else:
        data_folder = f"./data/{folder}/random_fixedTrue_{n_pins}pins_{28}by{28}pixels_{r}radius_{seed}seed"


transform = transforms.Compose([
    ToTensor(),         # Convert to tensor (as you were doing)
    Resize()  # Resize to 100x100
])

transformed_dataset = PinDataset(csv_file=f"{data_folder}/pins.csv",
                                      root_dir=f"./data/{dataset}/images/",
                                      transform=transform)

dataset_size = len(transformed_dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(
    transformed_dataset, [train_size, val_size, test_size]
)

# Create your DataLoader with the custom_collate_fn
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn, drop_last=True)

In [8]:
start_time = time.time()

r_dim, z_dim, h_dim = np_config.r_dim, np_config.z_dim, np_config.h_dim
if (config['experiment_id'] == 0): # Training

    neural_processes = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
    optimizer = torch.optim.Adam(neural_processes.parameters(), learning_rate)
#     lr_finder_NP =NPLRFinder(neural_processes, optimizer, device=device)
#     lr_finder_NP.find_lr(train_loader,input_channel=input_channel, start_lr=1e-5, end_lr=1, num_iter=20)
#     best_lr_NP = lr_finder_NP.find_best_lr()
#     print(f"Best Learning Rate for NP: {best_lr_NP}")

#     config['best_lr'] = best_lr_NP
    config['best_lr'] = 1e-4
    # Run and save the pipeline data
    test_losses, r2, experiment_id = run_and_save_pipeline_np(train_loader, val_loader, test_loader,\
                                               input_channel, epochs, val_every_epoch, config, num_runs, device)

else: # Testing
    experiment_id = config['experiment_id']
    if not os.path.exists(f'./results/neural_processes/{experiment_id}'):
        raise Exception(f"Could not find experiment with id: {experiment_id}")
    else:
        neural_processes = NeuralProcessImg(r_dim, z_dim, h_dim).to(device)
        try:
            neural_processes.load_state_dict(torch.load(f'./results/neural_processes/{experiment_id}/best_model_NP.pth'))
        except:
            raise Exception("The model you provided does not correspond with the selected architecture. Please revise and try again.")
        
        
        filename = f"test_{folder}_{partial_percent}"
        with open(f"./results/neural_processes/{experiment_id}/{filename}", "w") as f:
            f.write(f"MSE {best_MSE_test_loss}; NPP {best_NPP_test_loss}, {GP_best_NPP_test_loss} (GP)")
            
end_time = time.time()
elapsed_time = end_time - start_time
print("Time elapsed:", elapsed_time, "seconds")

Epoch: 0, Avg_loss: 402.063
Epoch: 0, Val_loss 113.709
Epoch: 1, Avg_loss: 83.094
Epoch: 2, Avg_loss: 47.212


KeyboardInterrupt: 

In [10]:
!jupyter kernelspec list

Available kernels:
  dsk            /home/shi.cheng/.local/share/jupyter/kernels/dsk
  jax            /home/shi.cheng/.local/share/jupyter/kernels/jax
  local-venv     /home/shi.cheng/.local/share/jupyter/kernels/local-venv
  myenv          /home/shi.cheng/.local/share/jupyter/kernels/myenv
  pycox          /home/shi.cheng/.local/share/jupyter/kernels/pycox
  python3        /home/shi.cheng/.local/share/jupyter/kernels/python3
  pytorch_env    /home/shi.cheng/.local/share/jupyter/kernels/pytorch_env
  survdata       /home/shi.cheng/.local/share/jupyter/kernels/survdata
  tf_env         /home/shi.cheng/.local/share/jupyter/kernels/tf_env
  tools          /home/shi.cheng/.local/share/jupyter/kernels/tools
